In [ ]:
!pip install -q transformers gradio torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

model_name = "deepset/xlm-roberta-large-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
def answer_question(context, question):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits) + 1

    answer_ids = inputs["input_ids"][0][answer_start:answer_end]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    start_prob = torch.softmax(start_logits, dim=1)[0][answer_start]
    end_prob = torch.softmax(end_logits, dim=1)[0][answer_end - 1]
    confidence = (start_prob * end_prob).item()

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    decoded_context = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

    if answer in decoded_context:
        highlighted_context = decoded_context.replace(answer, f"<mark><b>{answer}</b></mark>")
    else:
        highlighted_context = decoded_context

    return f" ਉੱਤਰ: {answer}\nਵਿਸ਼ਵਾਸ-ਪੱਧਰ: {confidence:.2f}", highlighted_context

In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(label="ਪੰਜਾਬੀ ਪੈਰਾ (Context)", lines=7, placeholder="ਇੱਥੇ ਪੈਰਾ ਦਿਓ..."),
        gr.Textbox(label="ਪ੍ਰਸ਼ਨ (Question)", placeholder="ਇੱਥੇ ਪ੍ਰਸ਼ਨ ਲਿਖੋ...")
    ],
    outputs=[
        gr.Textbox(label="ਉੱਤਰ & ਵਿਸ਼ਵਾਸ ਪੱਧਰ"),
        gr.HTML(label="ਪੈਰਾ (ਉੱਤਰ ਹਾਈਲਾਈਟ ਹੋਇਆ)")
    ],
    title="ਪੰਜਾਬੀ ਪ੍ਰਸ਼ਨ ਉੱਤਰਣ ਸਿਸਟਮ",
    description="ਪੰਜਾਬੀ ਵਿੱਚ ਸਵਾਲ ਪੁੱਛੋ ਅਤੇ ਉੱਤਰ ਪ੍ਰਾਪਤ ਕਰੋ!"
)

interface.launch(debug=True)